<a href="https://colab.research.google.com/github/LiamAMcNamara/HMMs-Lab3/blob/master/McNamara_Lab_3_HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #3: Hidden Markov Models**

Liam McNamara.308

June 11, 2020

---
---

**Part 1 (50 Points):** Implement the Viterbi Algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten). See elsner.hmm.xls for a bit more discussion on the problem. Remeber that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

There are 3 files provided for you that have observation/emission probabilities, transition probabilities, and test sequence data (for evaluation), respectively.

Please read the probabilities and observations from their Carmen URLs, *do not hard-code the values*. Refer back to lab 2 for exampes of working with CSV files and how to download them from URLs.

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label). The columns sum up to 1.

The test data has one line per sequence. When a sequence is less than five observations long, the last  columns are filled with zeros.



Set up the environment

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Read in the provided data from Carmen URL

In [2]:
emissionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
testData_dataframe = pd.read_csv(testDataURL)

Print out the top of the dataframes to make sure the data loaded correctly. The data tables should have the following format:

Emission Probabilities [3, 3]: 3 columns (P(x|...), C, H), and 3 rows corresponding the the number of ice creams

Transition Probabilities [3, 4]: 4 columns (P(x|...), C, H, START), and 3 rows (C, H, STOP)

Test Sequence Data [10, 6]: 6 columns (SeqNumber, Obs1...5), and 10 rows corresponing the the sequence sample number.

Note: The test data has one line per sequence. When a sequence is less than five observations long, the last columns are filled with zeros.

In [8]:
print("Emission Probabilities")
display(emission_dataframe.shape)
display(emission_dataframe.head())
print('\n')
print("Transition Probabilities")
display(transition_dataframe.shape)
display(transition_dataframe.head())
print('\n')
print("Test Sequence Data")
display(testData_dataframe.shape)
display(testData_dataframe.head())

Emission Probabilities


(3, 3)

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657




Transition Probabilities


(3, 4)

,P(x|...),C,H,START
0,C,0.87,0.47,0.5
1,H,0.07,0.47,0.5
2,STOP,0.06,0.06,0.0




Test Sequence Data


(10, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3


Initialize matrix variables and shorthand for emission and transition arrays

In [30]:
m_matrix = np.zeros((5, 2)) #Sequence of length 5 with 2 possible hidden vairable values (H, C)
c_matrix = np.zeros((4, 2)) #Sequence of length 5-1 with 2 hidden variable values

# In our class notes we referred to the transition matrix as 'A'
# and the emission/observation matrix as 'B'. So I will be doing the same.
A = transition_dataframe.to_numpy()
A = np.delete(A, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (C, H, STOP)
B = np.array(emission_dataframe)
B = np.delete(B, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (1, 2 , 3)

print(A)
print(B)

[[0.87 0.47 0.5]
 [0.07 0.47 0.5]
 [0.06 0.06 0.0]]
[[6.407e-01 2.000e-04]
 [1.481e-01 5.341e-01]
 [2.122e-01 4.657e-01]]


Step 1: Initialize

In [ ]:
# use priors to set first row of M matrix
m_matrix[0,0] = 